In [19]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import os
import time

In [20]:
class UnderwaterFishDetector:
    def __init__(self, model_path=None):
        """
        Initialize the fish detector
        """
        self.model = None
        self.load_model(model_path)
        
        # Detection parameters
        self.confidence_threshold = 0.5
        self.nms_threshold = 0.4
        
        # Colors for bounding boxes
        self.colors = [(0, 255, 0), (255, 0, 0), (0, 0, 255), (255, 255, 0)]
        
    def load_model(self, model_path):
        """Load YOLO model"""
        try:
            if model_path and os.path.exists(model_path):
                print(f"Loading custom model from {model_path}")
                self.model = YOLO(model_path)
            else:
                print("Loading YOLOv8 pretrained model...")
                self.model = YOLO('yolov8n.pt')  # nano version for speed
        except Exception as e:
            print(f"Error loading model: {e}")
            print("Using YOLOv8 nano as fallback")
            self.model = YOLO('yolov8n.pt')
    
    def preprocess_frame(self, frame):
        """Preprocess frame for underwater conditions"""
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        
        # CLAHE
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        l = clahe.apply(l)
        
        enhanced = cv2.merge([l, a, b])
        enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)
        enhanced = self.white_balance_correction(enhanced)
        
        # Sharpen
        kernel = np.array([[-1,-1,-1],
                          [-1, 9,-1],
                          [-1,-1,-1]])
        enhanced = cv2.filter2D(enhanced, -1, kernel * 0.1)
        
        return enhanced
    
    def white_balance_correction(self, img):
        """Simple white balance correction"""
        result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        avg_a = np.average(result[:, :, 1])
        avg_b = np.average(result[:, :, 2])
        result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
        result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
        result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
        return result
    
    def detect_fish(self, frame):
        """Detect fish in the frame"""
        processed_frame = self.preprocess_frame(frame.copy())
        results = self.model(processed_frame, conf=self.confidence_threshold, verbose=False)
        
        detections = []
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes.xyxy.cpu().numpy()
                confidences = result.boxes.conf.cpu().numpy()
                class_ids = result.boxes.cls.cpu().numpy()
                
                for i, (box, conf, cls_id) in enumerate(zip(boxes, confidences, class_ids)):
                    x1, y1, x2, y2 = map(int, box)
                    class_name = self.model.names[int(cls_id)]
                    
                    detection = {
                        'bbox': (x1, y1, x2, y2),
                        'confidence': conf,
                        'class': class_name,
                        'class_id': int(cls_id)
                    }
                    detections.append(detection)
                    
                    color = self.colors[i % len(self.colors)]
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    label = f"{class_name}: {conf:.2f}"
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                    cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), 
                                (x1 + label_size[0], y1), color, -1)
                    cv2.putText(frame, label, (x1, y1 - 5), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        return detections, frame
    
    def run_detection(self):
        """Run real-time fish detection using webcam"""
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Could not open webcam")
            return
        
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        print("Starting fish detection... Press 'q' to quit, 's' to save")
        
        frame_count, fps_start_time = 0, time.time()
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (640, 640))
            detections, processed_frame = self.detect_fish(frame)
            
            frame_count += 1
            if frame_count % 30 == 0:
                fps_end_time = time.time()
                fps = 30 / (fps_end_time - fps_start_time)
                fps_start_time = fps_end_time
            else:
                fps = 0
            
            if fps > 0:
                cv2.putText(processed_frame, f"FPS: {fps:.1f}", (10, 30), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(processed_frame, f"Detections: {len(detections)}", (10, 60), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.imshow('Underwater Fish Detection', processed_frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                filename = f"fish_detection_{timestamp}.jpg"
                cv2.imwrite(filename, processed_frame)
                print(f"Frame saved as {filename}")
        
        cap.release()
        cv2.destroyAllWindows()
    
    def train_custom_model(self, dataset_path, epochs=100):
        """Train a custom YOLO model on fish dataset"""
        print("Training custom fish detection model...")
        
        config_content = f"""
path: {dataset_path}
train: train/images
val: valid/images
test: test/images
nc: 1
names: ['fish']
"""
        config_path = os.path.join(dataset_path, "dataset.yaml")
        with open(config_path, 'w') as f:
            f.write(config_content)
        
        model = YOLO('yolov8l.pt')
        results = model.train(
            data=config_path,
            epochs=epochs,
            imgsz=640,
            batch=16,
            name='fish_detection',
            save=True,
            cache=True,
            device=0 if torch.cuda.is_available() else 'cpu'
        )
        print("Training completed!")
        return results


In [21]:
def main():
    print("Underwater Fish Detection System")
    print("=" * 40)
    print("Dataset Structure Expected:")
    print("dataset_root/")
    print("├── train/images/, labels/")
    print("├── valid/images/, labels/")
    print("└── test/images/, labels/")
    print("=" * 40)
    
    detector = UnderwaterFishDetector()
    
    print("\nOptions:")
    print("1. Run real-time detection with webcam")
    print("2. Train custom model (requires dataset)")
    
    choice = input("\nEnter your choice (1-2): ")
    if choice == '1':
        detector.run_detection()
    elif choice == '2':
        dataset_path = input("Enter path to dataset directory: ")
        if os.path.exists(dataset_path):
            epochs = int(input("Enter number of epochs (default 100): ") or "100")
            detector.train_custom_model(dataset_path, epochs)
        else:
            print("Dataset path does not exist!")
    else:
        print("Invalid choice!")


In [5]:
main()

NameError: name 'main' is not defined

In [4]:
from ultralytics import YOLO

# Load trained model
model = YOLO("D:\\Project\\undwerwater-object-detection\\best.pt")

# Run on a folder of images
results = model.predict(source=r"D:\Project\undwerwater-object-detection\fish-dataset\fish-dataset\test\images", conf=0.5, save=True)



image 1/59 D:\Project\undwerwater-object-detection\fish-dataset\fish-dataset\test\images\frame1008_jpg.rf.f0b0942e2c7d4e83fb55492060d6da1b.jpg: 640x640 19 fishs, 75.8ms
image 2/59 D:\Project\undwerwater-object-detection\fish-dataset\fish-dataset\test\images\frame1126_jpg.rf.6ce0aef287cdb7f6b8b65fd2178ef637.jpg: 640x640 10 fishs, 67.5ms
image 3/59 D:\Project\undwerwater-object-detection\fish-dataset\fish-dataset\test\images\frame1133_jpg.rf.b4be115c688f2e3b65883180aa034f35.jpg: 640x640 21 fishs, 61.8ms
image 4/59 D:\Project\undwerwater-object-detection\fish-dataset\fish-dataset\test\images\frame1188_jpg.rf.6c85ef65438c5884661129e1da5da478.jpg: 640x640 16 fishs, 61.9ms
image 5/59 D:\Project\undwerwater-object-detection\fish-dataset\fish-dataset\test\images\frame1239_jpg.rf.2ff6118ab6051e6a8f9c74a65d2963d5.jpg: 640x640 17 fishs, 61.0ms
image 6/59 D:\Project\undwerwater-object-detection\fish-dataset\fish-dataset\test\images\frame1241_jpg.rf.920957b398a5a92d86d20626ae11be99.jpg: 640x640 16

In [3]:
def main():
    print("Underwater Fish Detection System")
    print("=" * 40)
    print("Dataset Structure Expected:")
    print("dataset_root/")
    print("├── train/images/, labels/")
    print("├── valid/images/, labels/")
    print("└── test/images/, labels/")
    print("=" * 40)
    
    detector = UnderwaterFishDetector(model_path="runs/detect/fish_detection5/weights/best.pt")
    
    print("\nOptions:")
    print("1. Run real-time detection with webcam")
    print("2. Train custom model (requires dataset)")
    
    choice = input("\nEnter your choice (1-2): ")
    if choice == '1':
        detector.run_detection()
    elif choice == '2':
        dataset_path = input("Enter path to dataset directory: ")
        if os.path.exists(dataset_path):
            epochs = int(input("Enter number of epochs (default 100): ") or "100")
            detector.train_custom_model(dataset_path, epochs)
        else:
            print("Dataset path does not exist!")
    else:
        print("Invalid choice!")

In [ ]:
main()